# FODO optics

This will demonstrate how to scan symmetrically and asymetrically the quadruople strengths in a standard FODO lattice.

Later we will optimize for particular average beta function.

Finally, we will track a beam and gather statistics from the particles.

In [ ]:
from pytao import Tao
import numpy as np
import matplotlib.pyplot as plt
import os

%config InlineBackend.figure_format = 'retina'

In [ ]:
tao = Tao(
    "-init $ACC_ROOT_DIR/bmad-doc/tao_examples/fodo/tao.init -lat $ACC_ROOT_DIR/bmad-doc/tao_examples/fodo/fodo.bmad -noplot"
)

In [ ]:
def add_info(d):
    twiss1 = tao.ele_twiss("q1")
    twiss2 = tao.ele_twiss("q2")

    d["mean_beta_a"] = (twiss1["beta_a"] + twiss2["beta_a"]) / 2
    d["mean_beta_b"] = (twiss1["beta_b"] + twiss2["beta_b"]) / 2
    d["phi_a"] = twiss2["phi_a"]
    d["phi_b"] = twiss2["phi_b"]
    return d

In [ ]:
%%tao
sho lat

## Symmetric FODO

In [ ]:
def set_kx(k1):
    cmds = [f"set ele q1 k1 = {k1}", f"set ele q2 k1 = {-k1}"]

    d = {}
    try:
        tao.cmds(cmds)
        tao.cmd("set global lattice_calc_on = T")
        d["good"] = True
        add_info(d)
    except RuntimeError:
        d["good"] = False

    return d


x = set_kx(1.4142136e01)
KEYS = x.keys()
x

In [ ]:
# Scan k1
n1 = 20
qvec1 = np.linspace(1, 25, n1)

RESULTS = []

# tao.cmd('set global plot_on = F')
for k in qvec1:
    res = set_kx(k)
    RESULTS.append(res)
# tao.cmd('set global plot_on = T')

In [ ]:
# Reshape data
DAT = {}
for key in KEYS:
    print(key)
    x = []
    for res in RESULTS:
        if key in res:
            x.append(res[key])
        else:
            x.append(np.nan)
    DAT[key] = np.array(x)

In [ ]:
DAT.keys()

In [ ]:
for key in KEYS:
    plt.plot(qvec1, DAT[key])
    plt.ylabel(key)
    plt.xlabel(r"k1 (m$^{-2}$)")
    plt.show()

In [ ]:
%%tao
sho dat

# Asymmetric FODO

Scan k1 for each quad

In [ ]:
def set_k(k1, k2):
    cmds = [f"set ele q1 k1 = {k1}", f"set ele q2 k1 = {-k2}"]

    d = {}
    try:
        tao.cmds(cmds)
        tao.cmd("set global lattice_calc_on = T")
        d["good"] = True
        add_info(d)
    except RuntimeError:
        d["good"] = False

    return d


x = set_k(1.4142136e01, 1.4142136e01)
KEYS = x.keys()
x

In [ ]:
set_k(1, 1)

In [ ]:
n1 = 50
n2 = 60
qvec1 = np.linspace(1, 15, n1)
qvec2 = np.linspace(1, 15, n2)
K1, K2 = np.meshgrid(qvec1, qvec2, indexing="ij")

fK1 = K1.flatten()
fK2 = K2.flatten()

In [ ]:
%%time
# Make data

tao.cmd("set global plot_on = F")

RESULTS = []
for k1, k2 in zip(fK1, fK2):
    res = set_k(k1, k2)
    #    print(res)
    RESULTS.append(res)


# tao.cmd('set global plot_on = T')

In [ ]:
# Reshape data
DAT = {}
for key in RESULTS[0]:
    print(key)
    x = []
    for res in RESULTS:
        if key in res:
            x.append(res[key])
        else:
            x.append(np.nan)

    DAT[key] = np.array(x).reshape(n1, n2)

# Plots

In [ ]:
NICE = {}
NICE["mean_beta_a"] = r"$<\beta_x>$"
NICE["mean_beta_b"] = r"$<\beta_y>$"


def nice(key):
    if key in NICE:
        return NICE[key]
    return key

In [ ]:
# fig, ax = plt.subplots(figsize=(10,8))


def plot1(key):
    plt.imshow(
        DAT[key],
        origin="lower",
        extent=[qvec1.min(), qvec1.max(), qvec2.min(), qvec2.max()],
        cmap="jet",
        vmax=10,
    )
    plt.xlabel("Q1 (+)k1 (1/m$^2$)")
    plt.ylabel("Q2 (-)k1 (1/m$^2$)")
    plt.colorbar(label=nice(key))
    plt.show()


plot1("mean_beta_a")
plot1("mean_beta_b")

# Optimize for some special beta functions

In [ ]:
def optimize(beta_a, beta_b):
    cmds = f"""
alias setbetas
veto var *
set lattice model=design
veto dat *
use dat fodo.betas[1,2]
use dat fodo.stability
set dat fodo.betas[1]|meas={beta_a}
set dat fodo.betas[2]|meas={beta_b}
use var quad
run
show var -bmad -good
    """
    lines = tao.cmds(
        cmds.split("\n"),
        suppress_lattice_calc=False,
        suppress_plotting=False,
        raises=False,
    )

    # Twiss at Q1
    T = tao.ele_twiss("Q1")
    return T


optimize(10, 20)

In [ ]:
# Check merit
tao.merit()

In [ ]:
# Check that the optimization worked
average_beta_a = tao.data("fodo", "betas", dat_index=1)["model_value"]
average_beta_b = tao.data("fodo", "betas", dat_index=2)["model_value"]
average_beta_a, average_beta_b

In [ ]:
# These are the K
kq1 = tao.ele_gen_attribs("Q1")["K1"]
kq2 = tao.ele_gen_attribs("Q2")["K1"]
kq1, kq2

# Alternative method: alias

A 'simple' Tao alias can be useful when running on the command line.


In [ ]:
tao.cmd(
    "alias setbetas veto var *;veto dat *;use datafodo.stability;use dat fodo.betas[1,2];set dat fodo.betas[1]|meas=[[1]];set dat fodo.betas[2]|meas=[[2]];use var quad;run;show var -bmad -good"
)
# tao.cmd('call SetBetas.tao', raises=False)

lines = tao.cmd("setbetas 40 25", raises=False)
lines[-3:]
tao.merit()

In [ ]:
T = tao.ele_twiss("Q1")
T

# Beam tracking 

Here we will make a new lattice with 10 cells that calls the single fodo lattice.

In [ ]:
from pytao.misc.markers import make_markers

In [ ]:
?make_markers

In [ ]:
smax = 20.0  # m

# Alternatively, if the lattice were already loaded
# smax = tao.lat_list('*', who='ele.s').max()

slist = np.linspace(0, smax, 200)

make_markers(slist, filename="markers.bmad")
smax

In [ ]:
# Make a lattice and write to a local file

latfile = os.path.join(os.getcwd(), "fodo10.bmad")

LAT2 = f"""

call, file = $ACC_ROOT_DIR/bmad-doc/tao_examples/fodo/fodo.bmad
call, file = markers.bmad

Q1[k1] = {kq1}
Q2[k1] = {kq2}

lat: line = (10*fodo1)

use, lat

"""
open(latfile, "w").write(LAT2)

In [ ]:
# Run with this lattice
tao = Tao(
    f"-init $ACC_ROOT_DIR/bmad-doc/tao_examples/fodo/tao.init -lat {latfile} -noplot"
)

In [ ]:
f"-init $ACC_ROOT_DIR/bmad-doc/tao_examples/fodo/tao.init -lat {latfile} -noplot"

In [ ]:
# Toggle the beam on and off
tao.cmd("set beam_init n_particle = 1000")
tao.cmd('set beam track_start = beginning')
tao.cmd('set beam track_end = end')
tao.cmd("set global track_type = beam;set global track_type = single")

## Get particles 

In [ ]:
import h5py
from pmd_beamphysics import ParticleGroup, particle_paths

with h5py.File("beam_dump.h5", "r") as h5:
    pp = particle_paths(h5)
    Plist = [ParticleGroup(h5[g]) for g in pp]

## Pretty plot

Traces can be made by gathering the coordinate arrays

In [ ]:
skip = 1  # make larger for faster plotting
fig, axes = plt.subplots(2, figsize=(12, 8))

axes[0].plot(
    [P.t[::skip] * 299792458 for P in Plist],
    [P.x[::skip] * 1e6 for P in Plist],
    alpha=0.01,
    color="black",
)

axes[1].plot(
    [P.t[::skip] * 299792458 for P in Plist],
    [P.y[::skip] * 1e6 for P in Plist],
    alpha=0.01,
    color="black",
)

axes[0].set_ylabel(r"$x$ (µm)")
axes[1].set_ylabel(r"$y$ (µm)")

axes[1].set_xlabel(r"$ct$ (m)")

for ax in axes:
    ax.set_ylim(-2000, 2000)

## Get some statistics

In [ ]:
k1 = "sigma_x"
k2 = "sigma_y"

x = np.array([P["mean_t"] * 299792458 for P in Plist])
y1 = np.array([P[k1] for P in Plist])
y2 = np.array([P[k2] for P in Plist])

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(x, y1 * 1e6, label=k1)
ax.plot(x, y2 * 1e6, label=k2)
ax.set_xlabel("<ct> (m)")
ax.set_ylabel(f"{k1}, {k2} (µm)")
plt.legend()

## Cleanup

In [ ]:
# Cleanup
!rm beam_dump.h5
!rm {latfile}
!rm markers.bmad